In [1]:
from __future__ import absolute_import, division, print_function

import math

import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.layers import Dense, Input, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping

print(tf.__version__)

2.4.1


In [2]:
# Define default plot styles  

from matplotlib import rc
import matplotlib.font_manager

rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

plot_style_0 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'linestyle': '--',
    'density': False
}

plot_style_1 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'density': False
}

plot_style_2 = {'alpha': 0.5, 'density': False}

In [3]:
# use pyroot for I/O
import ROOT
from ROOT import TCanvas, TPad, TFile, TPaveText
from ROOT import gBenchmark, gStyle, gROOT
def getarray_selected(filename):
    f = TFile(filename)
    tree = gROOT.FindObject('selectedEvents')
    array = tree.AsMatrix(columns=[ "dptt_rec","pN_rec","daT_rec","muMomRec","muCosThetaRec","piMomRec","piCosThetaRec","pMomRec","pCosThetaRec","cutBranch","weight",
                                    "topology","target","dptt_truth","pN_truth","daT_truth",
                                    "muMomTrue","muCosThetaTrue","piMomTrue","piCosThetaTrue","pMomTrue","pCosThetaTrue","Enutrue"])
    #array11=array[array[:,8]==1]
    array11=array
    #array11=array11[array11[:,9]==1]
    #array11=array11[:,0:7]
    array11[:,0]=array11[:,0]/10**3 # unit conversion to make all numbers to be O(1)
    array11[:,1]=array11[:,1]/10**3
    array11[:,2]=array11[:,2]/180
    array11[:,3]=array11[:,3]/10**3
    array11[:,5]=array11[:,5]/10**3
    array11[:,7]=array11[:,7]/10**3
    array11[:,9]=array11[:,9]/5
    #nGenEv = 40000000 # total number of events generated in the file, to calculate proper xsec normalization
    return array11#,array[0,7]/nGenEv
selected_neut6T = getarray_selected('/hepstore/kmtsui/T2K/work/xsLLhFitter_super/inputs/neut_prod6T_allTKI_geomfixed.root')
selected_genie = getarray_selected('/hepstore/kmtsui/T2K/work/xsLLhFitter_super/inputs/genie_allTKI_geomfixed.root')

Welcome to JupyROOT 6.22/09


In [4]:
selected_genie[0]

array([-4.07499023e-01,  1.48850879e+00,  9.11690776e-01,  4.98728564e+00,
        9.78087664e-01,  7.39538269e-01,  9.63274121e-01,  8.49755554e-01,
        8.00695479e-01,  6.00000000e-01,  1.14200912e-01,  8.00000000e+00,
        6.00000000e+00, -9.99000000e+02, -9.99000000e+02, -9.99000000e+02,
        4.29008252e+03,  9.78433311e-01, -9.99000000e+02, -9.99000000e+02,
        8.67040894e+02,  9.45809126e-01,  8.58656934e+03])

In [ ]:
# set up the DNN model for training
ndim=10
inputs = Input((ndim, )) # 7d vector as input
hidden_layer_1 = Dense(50, activation='relu')(inputs)
dropout_layer_1 = Dropout(0.2)(hidden_layer_1)
hidden_layer_2 = Dense(50, activation='relu')(dropout_layer_1)
dropout_layer_2 = Dropout(0.2)(hidden_layer_2)
hidden_layer_3 = Dense(50, activation='relu')(dropout_layer_2)
dropout_layer_3 = Dropout(0.2)(hidden_layer_3)
outputs = Dense(1, activation='sigmoid')(dropout_layer_3)

model = Model(inputs=inputs, outputs=outputs)

earlystopping = EarlyStopping(patience=10, 
                              verbose=1,
                              restore_best_weights=True)

In [ ]:
from tensorflow_addons.optimizers import AdamW

#just use a fraction of data for training
#nevents = 10**6
batch_size_defined = 1000

x_data_and_MCback = np.concatenate([selected_neut6T[:,0:ndim],selected_genie[:,0:ndim]])
    
y_data_and_MCback = np.concatenate([np.zeros(len(selected_neut6T[:,0:ndim])),
                                    np.ones(len(selected_genie[:,0:ndim]))])
    
W_data_and_MCback = np.concatenate([ selected_neut6T[:,ndim], selected_genie[:,ndim] ])

X_train_1, X_test_1, Y_train_1, Y_test_1, w_train_1, w_test_1 = train_test_split(
    x_data_and_MCback, y_data_and_MCback, W_data_and_MCback)


optimizer = AdamW(weight_decay=0.001,learning_rate=0.001)

model.compile(loss='binary_crossentropy',
              optimizer= optimizer,
              metrics=['accuracy'])


model.fit(X_train_1,
          Y_train_1,
          sample_weight=w_train_1,
          epochs=200,
          batch_size=batch_size_defined,
          validation_data=(X_test_1, Y_test_1, w_test_1),
          callbacks=[earlystopping],
          verbose=1)


In [ ]:
# from NN (DCTR)
def reweight(events):
    f = model.predict(events, batch_size=batch_size_defined)
    weights = f / (1. - f)
    weights[weights>10]=10
    return np.squeeze(np.nan_to_num(weights))

In [ ]:
# get the reweight value
w_model = reweight(array11_Eb0[:,0:ndim])

In [ ]:
w_model

In [ ]:
# make plots
for plotNum in range(7):

        w_norm=1./(np.sum(w_model)/len(w_model))*w27*len(array11_Eb27)/w0/len(array11_Eb0) #properly weight to normalize the reweighted events

        bins_Enu = np.linspace(0.4, 6, 41)
        bins_mumom = np.linspace(0, 6, 41)
        bins_mutheta = np.linspace(-1, 1, 41)
        bins_pimom = np.linspace(0, 1.500, 41)
        bins_pitheta = np.linspace(-1, 1, 41)
        bins_mupitheta = np.linspace(-1, 1, 41)
        bins_q2 = np.linspace(0, 2, 21)

        xlabel = (r"$E_\nu (MeV)$",r"$p_\mu (MeV)$",r"$\cos\theta_\mu$",r"$p_\pi (MeV)$",r"$\cos\theta_\pi$",r"$\cos\theta_{\mu\pi}$",r"$Q2 (GeV^2)$")

        bins_array = (bins_Enu,bins_mumom,bins_mutheta,bins_pimom,bins_pitheta,bins_mupitheta,bins_q2)
        legend_loc = ('lower right','center right','center left','center left','center left','center left','center right')

        bins_w= np.linspace(0, 10, 101)

        fig, ax = plt.subplots(2,
                            2,
                            figsize=(12, 10),
                            constrained_layout=True,
                            #sharey='row'
                            )

        binwidth = bins_array[plotNum][1]-bins_array[plotNum][0]

        ax[0,0].set_xlabel(xlabel[plotNum])
        ax[0,0].set_ylabel(r'$d\sigma/dvar$')
        h1=ax[0,0].hist(array11_Eb0[:,plotNum], bins=bins_array[plotNum], **plot_style_2, label='Eb=0',weights=w0/binwidth*np.ones(len(array11_Eb0)),color="blue")
        h2=ax[0,0].hist(array11_Eb27[:,plotNum], bins=bins_array[plotNum], **plot_style_2, label='Eb=27MeV',weights=w27/binwidth*np.ones(len(array11_Eb27)),color="red")
        legend = ax[0,0].legend(
                    title='NuWro MC mode11',
                    loc=legend_loc[plotNum],
                    frameon=False)
        plt.setp(legend.get_title(), multialignment='center')
        ax2 = ax[0,0].twinx()
        ax2.set_ylabel('ratio')
        bincenter = 0.5 * (h1[1][1:] + h1[1][:-1])
        plt.errorbar(bincenter, h1[0]/h1[0], fmt='-', color="blue")
        plt.errorbar(bincenter, h2[0]/h1[0], fmt='-', color="red")

        ax[0,1].set_xlabel(xlabel[plotNum])
        ax[0,1].set_ylabel(r'$d\sigma/dvar$')
        #ax[0,1].hist(array11_Eb0[:,2], bins=bins_mutheta, **plot_style_2, label='Eb=0',weights=array11_Eb0[:,6]/40000000/0.05)
        #ax[0,1].hist(array11_Eb27[:,2], bins=bins_mutheta, **plot_style_2, label='Eb=27MeV',weights=array11_Eb27[:,6]/40000000/0.05)
        h3=ax[0,1].hist(array11_Eb0[:,plotNum], bins=bins_array[plotNum], **plot_style_2, label='Eb=0',weights=w0/binwidth*np.ones(len(array11_Eb0)),color="blue")
        h4=ax[0,1].hist(array11_Eb0[:,plotNum], bins=bins_array[plotNum], **plot_style_2, label='Eb=27MeV (Reweight)',
                        weights=w0/binwidth*np.double(w_model)*w_norm,color="orange")
        ax4 = ax[0,1].twinx()
        ax4.set_ylabel('ratio')
        plt.errorbar(bincenter, h3[0]/h3[0], fmt='-', color="blue")
        plt.errorbar(bincenter, h4[0]/h3[0], fmt='-', color="orange")
        plt.errorbar(bincenter, h2[0]/h1[0], fmt='-', color="red")
        plt.errorbar(bincenter,  h4[0]/h3[0]/(h2[0]/h1[0]), fmt='-', color="black",label="Ratio of Ratio")
        legend = ax[0,1].legend(
                    title='NuWro MC mode11',
                    loc=legend_loc[plotNum],
                    frameon=False)
        ax4.legend(loc='upper right',frameon=False)
        plt.setp(legend.get_title(), multialignment='center')

        ax[1,0].set_xlabel('reweight value')
        ax[1,0].hist(w_model, bins=bins_w, **plot_style_2, label='Eb=0',color="blue")

        ax[1,1].set_xlabel('reweight value')
        ax[1,1].set_ylabel(xlabel[plotNum])
        ax[1,1].hist2d(w_model, array11_Eb0[:,plotNum], bins=(bins_w,bins_array[plotNum]))

        #ax[2,0].hist2d(array11_Eb0[:,0], array11_Eb0[:,1], bins=(bins_mumom,bins_mumom))

        #ax[2,1].hist2d(array11_Eb27[:,0], array11_Eb27[:,1], bins=(bins_mumom,bins_mumom))

        fig.show()